**Creating a 3D array of size (2, 9, 5)**
```python
array_3d = np.array([
    [
        [1, 2, 3, 4, 5],
        [6, 7, 8, 9, 10],
        [41, 42, 43, 44, 45]
    ],
    [
        [46, 47, 48, 49, 50],
        [81, 82, 83, 84, 85],
        [86, 87, 88, 89, 90]
    ]
])
```

**Get flatten array from index [:,:,1]**
```python
array_3d[:,:,1].flatten() 
```

[ 2  7 42 47 82 87]

=**Get flatten array from index [:,1,:]**
```python
array_3d[:,1,:].flatten()
```

[ 6  7  8  9 10 81 82 83 84 85]

**Get flatten array from index [1,:,:]**
```python
array_3d[1,:,:].flatten()
```

[46 47 48 49 50 81 82 83 84 85 86 87 88 89 90]


In [1]:
import {InferenceSession, Tensor, env} from 'onnxruntime-web';
const ndarray = require('ndarray')
const ops = require('ndarray-ops')
const fs = require('fs')
import Jimp from "jimp";
import { Niivue, NVImage } from "@niivue/niivue";
import { cv } from "opencv-wasm";


In [2]:
// in kernel, The configuration runs using node, and not in the browser,
// which is why window is not defined
// we need to define it to make the code work

// global.window = { navigator: { userAgent: 'node.js' } };

import crypto from "node:crypto";
global.crypto ??= crypto;
const volume = await NVImage.loadFromUrl({url:"https://niivue.github.io/niivue-demo-images/mni152.nii.gz"})
console.log(volume)

NVImage {
  DT_NONE: 0,
  DT_UNKNOWN: 0,
  DT_BINARY: 1,
  DT_UNSIGNED_CHAR: 2,
  DT_SIGNED_SHORT: 4,
  DT_SIGNED_INT: 8,
  DT_FLOAT: 16,
  DT_COMPLEX: 32,
  DT_DOUBLE: 64,
  DT_RGB: 128,
  DT_ALL: 255,
  DT_INT8: 256,
  DT_UINT16: 512,
  DT_UINT32: 768,
  DT_INT64: 1024,
  DT_UINT64: 1280,
  DT_FLOAT128: 1536,
  DT_COMPLEX128: 1792,
  DT_COMPLEX256: 2048,
  DT_RGBA32: 2304,
  name: 'mni152.nii.gz',
  id: '592b5bdd-4a35-493e-95f7-c24f12fd5413',
  _colormap: 'gray',
  _opacity: 1,
  percentileFrac: 0.02,
  ignoreZeroVoxels: false,
  trustCalMinMax: true,
  colormapNegative: '',
  colormapLabel: [],
  frame4D: 0,
  cal_minNeg: NaN,
  cal_maxNeg: NaN,
  colorbarVisible: true,
  visible: true,
  modulationImage: null,
  modulateAlpha: 0,
  series: [],
  onColormapChange: [Function (anonymous)],
  onOpacityChange: [Function (anonymous)],
  hdr: si {
    littleEndian: true,
    dim_info: 0,
    dims: [
      3, 207, 256, 215,
      1,   1,   1,   1
    ],
    intent_p1: 0,
    intent_p2: 0,


In [3]:
function getMax(arr) {
    let len = arr.length;
    let max = -Infinity;

    while (len--) {
        max = arr[len] > max ? arr[len] : max;
    }
    return max;
}

function getMin(arr) {
    let len = arr.length;
    let min = Infinity;

    while (len--) {
        min = arr[len] < min ? arr[len] : min;
    }
    return min;
}

In [4]:
function normalizeArray(
    array: Float32Array | Uint8Array,
    max: number
  ): Float32Array {
    let normalizedArray = new Float32Array(array.length);
    for (let i = 0; i < array.length; i++) {
      normalizedArray[i] = array[i] / max;
    }
    return normalizedArray;
  }
  
  function stackSliceToRGB(
    buffer: Float32Array | Uint8Array,
  ): Float32Array {
    let bufferLength = buffer.length,
      result = new Float32Array(bufferLength * 3);
  
    for (let i = 0; i < bufferLength; i++) {
      result[3 * i] = buffer[i];
      result[3 * i + 1] = buffer[i];
      result[3 * i + 2] = buffer[i];
    }
    return result;
  }
  
  

In [5]:
const preprocessVolume = (image) => {
    try {
      const imageRAS = image.img2RAS();
      // console.log("image.calMax, image.calMin ", image.global_max, image.global_min);
      const maxVal = getMax(imageRAS);
      const normalizedArray = normalizeArray(imageRAS, maxVal);
      console.log("normalizedArray ", normalizedArray.reduce((a, b) => a + b, 0));
      return normalizedArray;
    } catch (e) {
      console.log(`failed to preprocess volume: ${e}. `);
      throw Error(`failed to preprocess volume: ${e}. `);
    }
  }

const preprocess = (slice2D: Float32Array, sliceId: number, width: number, height: number): Float32Array => {
    try {
      // console.log("imageArray ", slice2D);
      let image3Channels: Float32Array;
      const sliceArray = slice2D.slice(
        width * height * sliceId,
        width * height * (sliceId + 1),
      );
      console.log("sliceArray ", sliceArray.reduce((a, b) => a + b, 0));
  
      image3Channels = stackSliceToRGB(sliceArray);
      console.log("image3Channels ", image3Channels, image3Channels.reduce((a, b) => a + b, 0));
      return image3Channels;
    } catch (e) {
      console.log(`failed to inference ONNX model: ${e}. `);
      throw Error(`failed to inference ONNX model: ${e}. `);
    }
};


In [11]:
let stack_processed = preprocess(volume.img2RAS(), 90, dims[0], dims[1]);

sliceArray  6478747
image3Channels  Float32Array(158976) [
  0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
  0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
  0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
  0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
  0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
  0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
  0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
  0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
  0, 0, 0, 0,
  ... 158876 more items
] 19436241


In [14]:
imagedataToImage(stack_processed, [dims[0], dims[1]]);

Uint8Array(10) [
  0, 0, 0, 0, 0,
  0, 0, 0, 0, 0
]
Mat {}


In [13]:
function imagedataToImage(imagedata: Float32Array | Uint8Array, dims: number[]) {
    let inputImage;
    // if (dims[0]*dims[1] !== imagedata.length) {
    //     throw new Error("Image data size does not match the dimensions");
    // }

      inputImage = new Uint8Array(dims[0]*dims[1]*3);
      for (let i=0; i<imagedata.length; i++) {
        inputImage[i] = imagedata[i]*255;
    }

    let dst = cv.matFromArray(dims[0], dims[1], cv.CV_8UC3, inputImage);
    console.log(dst.data.slice(600, 610));
    console.log(dst);
    new Jimp({
        width: dims[0],
        height: dims[1],
        data: Buffer.from(dst.data)
    }).write('./niivue.png');
}


In [97]:
function nvVolumetoSlice(volume, dims, fov, index): any {
    let sliceArray;
    switch (fov) {
        // sagittal dimsRAS[1], coronal dimsRAS[2], axial dimsRAS[3]
        
        // 1. if axial, slice the array from volume
        case "axial":
            sliceArray = volume.slice(
                dims[0] * dims[1] * index,
                dims[0] * dims[1] * (index + 1),
            );
            console.log("axial sliceArray",dims[1], dims[2], index, sliceArray.length);
        // 2. if coronal, 
        case "coronal":
        // 3. if sagittal,
        case "sagittal":
    }
    return sliceArray;
    
}

function nvSliceToTensor(nvSlice, dims): any {
    console.log("nvSlice", nvSlice.length);
    // 1. preprocess
    let image3Channels = stackSliceToRGB(nvSlice);
    // 2. convert to float32
    console.log("image3Channels", image3Channels.length);
    // 3. tensor
    const inputTensor = new Tensor("float32", image3Channels, dims);
    return inputTensor;
}

function imageDataToTensor(data, dims, max){
    // 1a. Extract the R, G, and B channels from the data to form a 3D int array
    const [R, G, B]: number[][] = [[], [], []];
    for (let i = 0; i < data.length; i += 3) {
      R.push(data[i]);
      G.push(data[i + 1]);
      B.push(data[i + 2]);
      // 2. skip data[i + 3] thus filtering out the alpha channel
    }
    // 1b. concatenate RGB ~= transpose [224, 224, 3] -> [3, 224, 224]
    const transposedData = R.concat(G).concat(B);
  
    // 3. convert to float32
    let i, l = transposedData.length; // length, we need this for the loop
    const float32Data = new Float32Array(dims[3]*dims[1]*dims[2]); // create the Float32Array for output
    for (i = 0; i < l; i++) {
      float32Data[i] = transposedData[i] / 252.0; // convert to float
    }
  
    const inputTensor = new Tensor("float32", float32Data, dims);
    return inputTensor;
  }

function arrayToTensor(array, dims){
    let float32Data = new Float32Array(array.length*3);
    for (let i = 0; i < array.length; i++) {
        float32Data[i] = array[i];
        float32Data[i + array.length] = array[i];
        float32Data[i + 2 * array.length] = array[i];
    }
    const inputTensor = new Tensor("float32", float32Data, dims);
    return inputTensor;
}

In [98]:
const dims = [volume.dimsRAS[1], volume.dimsRAS[2], volume.dimsRAS[3]];
let nvVolume = preprocessVolume(volume);
let nvSlice = nvVolumetoSlice(nvVolume, dims, "axial", 90);
let inputTensor = nvSliceToTensor(nvSlice, [1, 3, dims[0], dims[1]]);
const maxVal = getMax(volume.img2RAS());
let transposedTensor = imageDataToTensor(inputTensor.data, [1, 3, dims[0], dims[1]], maxVal);

normalizedArray  3269611.783972291
axial sliceArray 256 215 90 52992
nvSlice 52992
image3Channels 158976


In [100]:
inputTensor.data.slice(16000,16007)

Float32Array(7) [
  0.4246031641960144,
  0.4246031641960144,
  0.2936508059501648,
  0.2936508059501648,
  0.2936508059501648,
  0.1626984179019928,
  0.1626984179019928
]


In [80]:
// const dims = [volume.dimsRAS[1], volume.dimsRAS[2], volume.dimsRAS[3]];
// // let nvVolume = preprocessVolume(volume);
// let nvSlice = nvVolumetoSlice(volume.img2RAS(), dims, "axial", 90);
// // let inputImage = nvSliceToImage(nvSlice);
// const maxVal = getMax(volume.img2RAS());
// let inputTensor = arrayToTensor(nvSlice, [1, 3, dims[0], dims[1]], maxVal);

axial sliceArray 256 215 90 52992


In [87]:
imagedataToImage(inputTensor.data, [dims[0], dims[1]]);

Uint8Array(10) [
  0, 0, 0, 0, 0,
  0, 0, 0, 0, 0
]
Mat {}


In [95]:
console.log(inputTensor.data.reduce((a,b) => a+b,0), inputTensor.dims, getMax(inputTensor.data), getMin(inputTensor.data));

console.log(volume.img2RAS().reduce((a,b) => a+b, 0), getMax(volume.img), getMin(volume.img), volume.global_max, volume.global_min);

77127.94049503142 [ 1, 3, 207, 256 ] 0.932539701461792 0


823942169 252 0 80 40


In [20]:
global.self = global;
env.wasm.numThreads = 1;

const encoderSession = await InferenceSession.create('../public/model/efficient_sam_vitt_encoder.onnx');
const decoderSession = await InferenceSession.create('../public/model/efficient_sam_vitt_decoder.onnx');


In [51]:
async function runEncoder(model, preprocessedData): Promise<[Tensor, number]> {
    const start = new Date();
    try {
      const feeds: Record<string, Tensor> = {};
      feeds[model.inputNames[0]] = preprocessedData;
      const outputData = await model.run(feeds);
      const end = new Date();
      const inferenceTime = (end.getTime() - start.getTime());
      const output = outputData[model.outputNames[0]];
      return [output, inferenceTime];
    } catch (e) {
      console.error(e);
      throw new Error();
    }
  }

async function runDecoder(model, embedding, inputPoints, inputLabels, oriSize): Promise<[Tensor, number]> {
  const decoderStart = new Date();
  try {
    const decoderFeeds: Record<string, Tensor> = {};
    console.log("model.inputNames", model.inputNames)
    decoderFeeds[model.inputNames[0]] = embedding;
    decoderFeeds[model.inputNames[1]] = inputPoints;
    decoderFeeds[model.inputNames[2]] = inputLabels;
    decoderFeeds[model.inputNames[3]] = oriSize;

    // feeds[model.inputNames[1]] = new Tensor("float32", inputPoints, [1, 1, inputPoints.length/2, 2]);
    // feeds[model.inputNames[2]] = new Tensor("float32", new Array(inputPoints.length/2).fill(1), inputPoints.length/2);
    // feeds[model.inputNames[3]] = new Tensor("int64", oriSize);
    const outputData = await model.run(decoderFeeds);
    const decoderEnd = new Date();
    const decoderInferenceTime = (decoderEnd.getTime() - decoderStart.getTime());
    const decoderOutput = outputData[model.outputNames[0]];
    return [decoderOutput, decoderInferenceTime];
  } catch (e) {
    console.error(e);
    throw new Error();
  }
}

In [102]:
const [embedding, encoderTime] =  await runEncoder(encoderSession, inputTensor);
var encoderOutput = embedding.data;
var inferenceTime = encoderTime;
// var results = softmax(Array.prototype.slice.call(output))

In [73]:
console.log((encoderOutput as Float32Array).reduce((a,b) => a + b, 0), inferenceTime);

1811.537114978095 10161


In [104]:
let points = new Float32Array([34, 150, 35, 150])
let labels = new Float32Array(points.length/2).fill(1)
let pointsTensor = new Tensor("float32", points, [1, 1, 2, 2]);
let labelsTensor = new Tensor("float32", labels, [1,1,labels.length]);
let originalSize = new Tensor("int64", [dims[0], dims[1]], [2]);
const [mask, decoderTime] = await runDecoder(decoderSession, embedding, pointsTensor, labelsTensor, originalSize);
var decoderOutput = mask.data;
var inferenceTime = decoderTime;

model.inputNames [
  'image_embeddings',
  'batched_point_coords',
  'batched_point_labels',
  'orig_im_size'
]


In [116]:
function transposeChannel(data, dims) {
    let transposedData = new Float32Array(dims[3]*dims[4]*dims[2]);
    const channelLength = dims[3]*dims[4];
    for (let i = 0; i < data.length; i+=3) {
        transposedData[i] = data[i];
        transposedData[i+1] = data[i+channelLength];
        transposedData[i+2] = data[i+2*channelLength];
    }
    return transposedData;
}

In [115]:
decoderOutput

Float32Array(158976) [
  -0.35261034965515137,   -0.357004314661026, -0.14563128352165222,
   -0.1738205999135971, -0.20448988676071167, -0.21276701986789703,
  -0.20582811534404755, -0.21688145399093628, -0.18029552698135376,
  -0.19248846173286438, -0.19351686537265778,  -0.2097848504781723,
   -0.1618129163980484, -0.18173974752426147, -0.20316855609416962,
  -0.22348643839359283,  -0.1598932296037674, -0.19097548723220825,
   -0.2209727168083191,  -0.2352137416601181, -0.15838882327079773,
   -0.1931561976671219,  -0.2211531400680542, -0.23099099099636078,
  -0.15211433172225952,  -0.1923026442527771, -0.21370403468608856,
  -0.22706826031208038, -0.14434345066547394, -0.18436533212661743,
  -0.20926396548748016, -0.22062653303146362,  -0.1562342643737793,
  -0.18869848549365997, -0.21952533721923828,  -0.2312026172876358,
  -0.16751091182231903, -0.19559451937675476,  -0.2177271842956543,
  -0.24237291514873505, -0.18587234616279602, -0.21461135149002075,
  -0.22676952183246613,  

In [118]:
transposedMask

Float32Array(158976) [
  -0.35261034965515137, -31.931503295898438,  -4.792166709899902,
   -0.1738205999135971,  -34.34942626953125, -5.2030110359191895,
  -0.20582811534404755,  -34.80441665649414,  -5.148431301116943,
  -0.19248846173286438,  -33.82314682006836,  -4.973199367523193,
   -0.1618129163980484,  -33.84146499633789,  -4.893842697143555,
  -0.22348643839359283,  -34.55813217163086,  -5.150190830230713,
   -0.2209727168083191, -33.819557189941406,  -5.047060966491699,
   -0.1931561976671219, -32.258209228515625,  -4.824119567871094,
  -0.15211433172225952, -32.487911224365234,  -4.772770404815674,
  -0.22706826031208038, -33.720706939697266,  -5.021378040313721,
  -0.20926396548748016, -33.366371154785156,  -4.936467170715332,
  -0.18869848549365997,  -32.77333068847656, -4.7943806648254395,
  -0.16751091182231903,   -32.9053955078125, -4.6888251304626465,
  -0.24237291514873505,  -33.23194122314453,  -4.802729606628418,
  -0.22676952183246613,   -33.2132453918457, -4.76872

In [117]:
let transposedMask = transposeChannel(decoderOutput, mask.dims);

In [122]:
let maskImage = new Uint8Array(dims[0]*dims[1]*3);
for (let i=0; i<transposedMask.length; i++) {
    if (transposedMask[i] >= 0) {
        maskImage[i] = 255;
    } else {
        maskImage[i] = 0;
    }
}

In [123]:
imagedataToImage((maskImage).slice(dims[0]*dims[1]), [dims[0], dims[1]]);

Uint8Array(10) [
  0, 0, 0, 0, 0,
  0, 0, 0, 0, 0
]
Mat {}


In [57]:
(decoderOutput as Float32Array).reduce((a,b) => a + b, 0)

-1811852.2913827675
